# LangChain Cohere Agent using Tools

Install Libraries

In [6]:
# ! pip install streamlit
# ! pip install qdrant-client
# ! pip install langchain
# ! pip install langchain-community
# ! pip install langchain-qdrant
# ! pip install langchain-cohere
# ! pip install tiktoken
# ! pip install cohere
# ! pip install bs4

Imports for Tool Use

In [10]:
from IPython.display import HTML
import markdown
import warnings
import streamlit as st
import os
from langchain_cohere.llms import Cohere
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_cohere import ChatCohere, create_cohere_react_agent
from langchain.agents import AgentExecutor
from cohere_tools import get_internet_search_tool, get_wikipedia_tool, get_openweathermap_tool
from alpha_vantage_tools import (
    # get_search_symbol_tool,
    get_exchange_rate_tool,
    # get_time_series_daily_tool,
    # get_time_series_weekly_tool,
    # get_market_news_sentimen_tool,
    # get_top_gainers_losers_tool,
)

Ignore Warnings

In [11]:

warnings.filterwarnings("ignore")

Import API-Keys

In [12]:
ALPHAVANTAGE_API_KEY = st.secrets["ALPHAVANTAGE_API_KEY"]
os.environ["ALPHAVANTAGE_API_KEY"] = st.secrets["ALPHAVANTAGE_API_KEY"]

COHERE_API_KEY = st.secrets["COHERE_API_KEY"]
os.environ["COHERE_API_KEY"] = st.secrets["COHERE_API_KEY"]

QDRANT_HOST = st.secrets["QDRANT_HOST"]
QDRANT_API_KEY = st.secrets["QDRANT_API_KEY"]

OPEN_WEATHER_API_KEY = st.secrets["OPEN_WEATHER_API_KEY"]
os.environ["OPENWEATHERMAP_API_KEY"] = OPEN_WEATHER_API_KEY

POLYGON_API_KEY = st.secrets["POLYGON_API_KEY"]
os.environ["POLYGON_API_KEY"] = st.secrets["POLYGON_API_KEY"]

TAVILY_API_KEY = st.secrets["TAVILY_API_KEY"]
os.environ["TAVILY_API_KEY"] = st.secrets["TAVILY_API_KEY"]

Invoke ChatCohere

In [9]:
chat = ChatCohere()
messages = [HumanMessage(content="knock knock")]
print(chat.invoke(messages))

content="Who's there?" additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '8c7b69ad-1a4f-40aa-b365-910b679dc73f', 'token_count': {'input_tokens': 68, 'output_tokens': 4}} response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '8c7b69ad-1a4f-40aa-b365-910b679dc73f', 'token_count': {'input_tokens': 68, 'output_tokens': 4}} id='run-3c2fb0b9-aba8-4378-b81a-03a069da25b2-0'


Invoke ChatCohere using ChatPromptTemplate

In [10]:
prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")
chain = prompt | chat
chain.invoke({"topic": "bears"})

AIMessage(content='Why don’t bears wear socks? \n\nThey have bear feet!', additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '96aec7ab-7bb7-47c9-a38e-10d1ba5d99cf', 'token_count': {'input_tokens': 72, 'output_tokens': 15}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '96aec7ab-7bb7-47c9-a38e-10d1ba5d99cf', 'token_count': {'input_tokens': 72, 'output_tokens': 15}}, id='run-db8eb780-c8ee-4220-8bd5-75c9b3c37435-0')

Invoke Cohere

In [11]:
llm = Cohere()
print(llm.invoke("Summarize the theory of Kahnemann and Tversky"))

 The theory of Kahnemann and Tversky is a collection of cognitive biases that affect how people make decisions and form beliefs. The theory consists of a list of heuristics (mental shortcuts) that are used by humans to make decisions, and the cognitive biases that result from these heuristics.

One of the most well-known heuristics is the availability heuristic, which is the tendency for people to make judgments based on how easy it is to think of examples for or against a particular issue. It is also known as the availability bias, which is a cognitive bias that causes people to overestimate the likelihood of events that are more easy to recall than events that are less easy to recall. For instance, individuals are more likely to believe that siblings resemble each other and that emigration rates are higher in the destination country when this country is next door, simply because these statements seem more available or easier to bring to mind. 

Let me know if you'd like to know more 

Tavily Internet Search Tool

In [13]:
llm = ChatCohere()
internet_search = get_internet_search_tool()
prompt = ChatPromptTemplate.from_template("{input}")
agent = create_cohere_react_agent(llm, [internet_search], prompt)
agent_executor = AgentExecutor(agent=agent, tools=[internet_search], verbose=True)
agent_executor.invoke(
    {
        "input": "In which year had Palantir its IPO at the NYSE? Return the exact date and year of the IPO.",
    }
)



> Entering new AgentExecutor chain...

I will search for 'Palantir IPO NYSE date' and write an answer based on the information I find.
{'tool_name': 'internet_search', 'parameters': {'query': 'Palantir IPO NYSE date'}}
[{'url': 'https://www.cnn.com/2020/09/30/tech/palantir-ipo/index.html', 'content': 'The company began trading on the New York Stock Exchange under the ticker symbol "PLTR" just after 1:30pm ET at $10 a share, up from the reference price listed by the NYSE of $7.25 a share ...'}, {'url': 'https://investorplace.com/palantir-stock-pltr-ipo-ultimate-guide/', 'content': 'Palantir stock begins trading on the NYSE on Tuesday, Sept. 30 under the ticker "PLTR". The company will pursue a direct listing, rather than a traditional initial public offering (IPO).'}, {'url': 'https://www.investopedia.com/palantir-ipo-what-you-need-to-know-5080082', 'content': "(If you want to hear us talk about it, check out this week's Express podcast)\n475.8 million Class A shares can be sold today

{'input': 'In which year had Palantir its IPO at the NYSE? Return the exact date and year of the IPO.',
 'output': 'Palantir began trading on the New York Stock Exchange on Tuesday, 30 September 2020.',
 'citations': [CohereCitation(start=57, end=64, text='Tuesday', documents=[{'url': 'https://investorplace.com/palantir-stock-pltr-ipo-ultimate-guide/', 'content': 'Palantir stock begins trading on the NYSE on Tuesday, Sept. 30 under the ticker "PLTR". The company will pursue a direct listing, rather than a traditional initial public offering (IPO).', 'id': 'doc_1'}], document_ids={'doc_1'}),
  CohereCitation(start=66, end=83, text='30 September 2020', documents=[{'url': 'https://www.cnn.com/2020/09/30/tech/palantir-ipo/index.html', 'content': 'The company began trading on the New York Stock Exchange under the ticker symbol "PLTR" just after 1:30pm ET at $10 a share, up from the reference price listed by the NYSE of $7.25 a share ...', 'id': 'doc_0'}, {'url': 'https://investorplace.com/p

Wikipedia Tool

In [24]:
llm = ChatCohere()
prompt = ChatPromptTemplate.from_template("{input}")
wikipedia_tool = get_wikipedia_tool()
agent = create_cohere_react_agent(llm, [wikipedia_tool], prompt)
agent_executor = AgentExecutor(agent=agent, tools=[wikipedia_tool], verbose=True)
response = agent_executor.invoke(
    {
        "input": "Explain the theory of Kahnemann and Tversky",
    }
)

display(HTML(markdown.markdown(response["output"])))



> Entering new AgentExecutor chain...

I will search for 'Kahnemann and Tversky theory' and write an answer based on the results.
{'tool_name': 'wikipedia-tool', 'parameters': {'query': 'Kahnemann and Tversky theory'}}
Page: Daniel Kahneman
Summary: Daniel Kahneman (; Hebrew: דניאל כהנמן; March 5, 1934 – March 27, 2024) was an Israeli-American cognitive scientist best-known for his work on the psychology of judgment and decision-making. He is also known for his work in behavioral economics, for which he was awarded the 2002 Nobel Memorial Prize in Economic Sciences together with Vernon L. Smith. Kahneman's published empirical findings challenge the assumption of human rationality prevailing in modern economic theory. Kahneman became known as the "grandfather of behavioral economics."
With Amos Tversky and others, Kahneman established a cognitive basis for common human errors that arise from heuristics and biases, and developed prospect theory. In 2011, Kahneman was named by Foreign P

Run tool without agent

In [73]:
wikipedia_tool.run("David Beckham")

"Page: David Beckham\nSummary: David Robert Joseph Beckham  ( BEK-əm; born 2 May 1975) is an English former professional footballer, the president and co-owner of Inter Miami CF and co-owner of Salford City. Primarily a right winger and known for his range of passing, crossing ability and free-kick taking, Beckham has been hailed as one of the greatest and most recognisable midfielders of his generation, as well as one of the best free-kick takers of all time. He won 19 major trophies in his career, and is the first English player to win league titles in four different countries: England, Spain, the United States and France.\nBeckham's professional club career began with Manchester United, where he made his first-team debut in 1992 at age 17. With United, he won the Premier League title six times, the FA Cup twice, the FA Charity Shield twice, the Intercontinental Cup and the UEFA Champions League in 1999. He then played four seasons with Real Madrid, winning the La Liga championship i

Openweathermap Tool

In [22]:
llm = ChatCohere()
prompt = ChatPromptTemplate.from_template("{input}")
tools = [get_openweathermap_tool(llm), get_wikipedia_tool()]
agent = create_cohere_react_agent(tools=tools, llm=llm, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
response = agent_executor.invoke({"input": "How is the weather in Salvador da Bahia?"})



> Entering new AgentExecutor chain...

I will search for the weather in Salvador da Bahia and then write an answer.
{'tool_name': 'open_weather_map', 'parameters': {'location': 'Salvador da Bahia'}}
In Salvador da Bahia, the current weather is as follows:
Detailed status: broken clouds
Wind speed: 6.48 m/s, direction: 163°
Humidity: 78%
Temperature: 
  - Current: 26.22°C
  - High: 26.22°C
  - Low: 26.22°C
  - Feels like: 26.22°C
Rain: {}
Heat index: None
Cloud cover: 82%Relevant Documents: 0
Cited Documents: 0
Answer: In Salvador da Bahia, the weather is currently 26.22°C, with broken clouds, 78% humidity, and a wind speed of 6.48 m/s.
Grounded answer: In Salvador da Bahia, the weather is currently <co: 0>26.22°C</co: 0>, with <co: 0>broken clouds</co: 0>, <co: 0>78% humidity</co: 0>, and a <co: 0>wind speed of 6.48 m/s</co: 0>.

> Finished chain.


Access Output

In [23]:
display(HTML(markdown.markdown(response["output"])))

Multiple Tools

Multiple Tool Use
1. wiki-tool
2. open_weather_map
3. internet_search

In [5]:

llm = ChatCohere()
tools = [get_openweathermap_tool(llm), get_internet_search_tool(), get_wikipedia_tool()]

prompt = ChatPromptTemplate.from_template(
    "Use the right tool to answer the following question. If wikipedia_tool delivers no result use internet_search: {input}"
)
agent = create_cohere_react_agent(tools=tools, llm=llm, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

print(agent_executor.invoke({"input": "Who was George Washington?"}))
print(
    agent_executor.invoke(
        {"input": "What's the weather like in Mortugaba, Bahia, Brazil?"}
    )
)
print(agent_executor.invoke({"input": "Whats in the news today?"}))
print(
    agent_executor.invoke({"input": "How did the Microsoft stock perform last week?"})
)



> Entering new AgentExecutor chain...

I will use Wikipedia to answer this question.
{'tool_name': 'wikipedia-tool', 'parameters': {'query': 'Who was George Washington?'}}
Page: George Washington Carver
Summary: George Washington Carver (c. 1864 – January 5, 1943) was an American agricultural scientist and inventor who promoted alternative crops to cotton and methods to prevent soil depletion. He was one of the most prominent black scientists of the early 20th century.
While a professor at Tuskegee Institute, Carver developed techniques to improve types of soils depleted by repeated plantings of cotton. He wanted poor farmers to grow other crops, such as peanuts and sweet potatoes, as a source of their own food and to improve their quality of life. Under his leadership, the Experiment Station at Tuskegee published over forty practical bulletins for farmers, many written by him, which included recipes; many of the bulletins contained advice for poor farmers, including combating soil d

Alpha Vantage Tool

In [ ]:
llm = ChatCohere()

prompt = ChatPromptTemplate.from_template("{input}")

tools = [
    # get_search_symbol_tool(),
    get_exchange_rate_tool(),
    # get_time_series_daily_tool(),
    # get_time_series_weekly_tool(),
    # get_market_news_sentimen_tool(),
    # get_top_gainers_losers_tool(),
]

agent = create_cohere_react_agent(tools=tools, llm=llm, prompt=prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
# alpha_vantage = AlphaVantageAPIWrapper()
# alpha_vantage._get_exchange_rate("USD", "JPY")
# alpha_vantage._get_time_series_daily("MSFT")
# alpha_vantage._get_time_series_weekly("IBM")
# alpha_vantage.search_symbols("American Superconductor")
# alpha_vantage._get_market_news_sentiment("Microsoft")
# alpha_vantage._get_top_gainers_losers()
# response = agent_executor.invoke(
#     {"input": "What is the symbol for American Superconductor?"}
# )
response = agent_executor.invoke({"input": "How much is one Dollar in Euro?"})
# response = agent_executor.invoke({"input": "Who are the gainers and losers at the stock exchange?"})
# response = agent_executor.invoke({"input": "How is Microsoft performing historically?"})
# response = agent_executor.invoke({"input": "How did Apple perform the last time?"})
# response = agent_executor.invoke({"input": "What is in the news about Nvidia?"})

Imports for Embeddings and Qdrant

In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.vectorstores import Qdrant
from langchain_community.vectorstores import Qdrant
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.conversation.memory import ConversationSummaryMemory
import qdrant_client

Build Qdrant Vector Store

In [ ]:
# web_links = ["https://www.fielmann-group.com/ueber-uns/?",
#              "https://de.wikipedia.org/wiki/Fielmann",
#              "https://de.wikipedia.org/wiki/G%C3%BCnther_Fielmann",
#              "https://de.wikipedia.org/wiki/Marc_Fielmann"] 

web_links = ["https://python.langchain.com/v0.2/docs/introduction",
             "https://python.langchain.com/v0.2/docs/concepts",
             "https://python.langchain.com/v0.2/docs/integrations/tools",
             "https://python.langchain.com/v0.2/docs/integrations/vectorstores",
             "https://python.langchain.com/v0.2/docs/integrations/document_loaders",
             "https://python.langchain.com/v0.2/docs/integrations/retrievers",
             "https://python.langchain.com/v0.2/docs/integrations/vectorstores/qdrant",
             "https://python.langchain.com/v0.2/docs/integrations/providers/cohere/#react-agent",
             "https://de.wikipedia.org/wiki/Herz#:~:text=Das%20Herz%20(lateinisch%20Cor%2C%20griechisch,die%20Versorgung%20aller%20Organe%20sichert."]
 

loader = WebBaseLoader(web_links)
document=loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
texts = text_splitter.split_documents(document)

embeddings = CohereEmbeddings(model = "embed-english-v2.0")

qdrant = Qdrant.from_documents(
    document,
    embeddings,
    url=QDRANT_HOST,
    api_key=QDRANT_API_KEY,
    prefer_grpc=True,
    collection_name="rag_documents",
    force_recreate=True,
)


Load Qdrant Vector Store

In [19]:
client = qdrant_client.QdrantClient(
    url=QDRANT_HOST,
    api_key=QDRANT_API_KEY,
)
embeddings = CohereEmbeddings(model="embed-english-v2.0")
vector_store = Qdrant(
    client = client,
    collection_name = "rag_documents",
    embeddings = embeddings
)
print("connection established !")

connection established !


Use Qdrant Vector Store as RAG Retriever for Cohere

In [25]:
prompt_template = """
You are a bot to answer questions from a document.
    
You will be given a context of the conversation made so far followed by a question, 
give the answer to the question using the context. 
The answer should be short, straight and to the point. If you don't know the answer, reply that the answer is not available.

Context: {context}

Question: {question}
Answer:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain_type_kwargs = { "prompt" : PROMPT }

llm = ChatCohere()

template = (
        """Combine the chat history and follow up question into 
        a standalone question. 
        If chat hsitory is empty, use the follow up question as it is.
        Chat History: {chat_history}
        Follow up question: {question}"""
    )

prompt = PromptTemplate.from_template(template)

chain = ConversationalRetrievalChain.from_llm(     
    llm = llm,
    chain_type = "stuff",
    memory = ConversationSummaryMemory(llm = llm, memory_key='chat_history', input_key='question', output_key= 'answer', return_messages=True),
    retriever = vector_store.as_retriever(search_type="mmr"),
    condense_question_prompt = prompt,
    return_source_documents=False,
    combine_docs_chain_kwargs=chain_type_kwargs,
)
customer_prompt = "What are tools in LangChain?"


llm_response = chain.invoke(
                {
                    "context": chain.memory.buffer,
                    "question": customer_prompt,
                },
                return_only_outputs=True,
            )

display(HTML(markdown.markdown(llm_response['answer'])))


In [ ]:
import yfinance as yf
def get_ticker_info(ticker_symbol: str) -> str:
    """"Gets the information about a stock market ticker symbol"""
    ticker = yf.Ticker(ticker_symbol)
    return ticker.info

In [26]:
import yfinance as yf

msft = yf.Ticker("MSFT")

# get all stock info
msft.info

{'address1': 'One Microsoft Way',
 'city': 'Redmond',
 'state': 'WA',
 'zip': '98052-6399',
 'country': 'United States',
 'phone': '425 882 8080',
 'website': 'https://www.microsoft.com',
 'industry': 'Software - Infrastructure',
 'industryKey': 'software-infrastructure',
 'industryDisp': 'Software - Infrastructure',
 'sector': 'Technology',
 'sectorKey': 'technology',
 'sectorDisp': 'Technology',
 'longBusinessSummary': 'Microsoft Corporation develops and supports software, services, devices and solutions worldwide. The Productivity and Business Processes segment offers office, exchange, SharePoint, Microsoft Teams, office 365 Security and Compliance, Microsoft viva, and Microsoft 365 copilot; and office consumer services, such as Microsoft 365 consumer subscriptions, Office licensed on-premises, and other office services. This segment also provides LinkedIn; and dynamics business solutions, including Dynamics 365, a set of intelligent, cloud-based applications across ERP, CRM, power 